## 音声データ文字起こし

1.   事前にGoogleDrive内に音声認識したい動画を格納してください
    *   直接Colabにアップロードする場合も考えましたが、遅いのでDRIVEから参照する形にしています。

2.   ランタイムのタイプをGPUに変更
    *   このファイルではデフォルトでGPU設定されています。

3.   コードを実行する(初めは時間がかかります)

4.   対象のファイル、フォルダのPATHを入力する。
    *   PATHはGoogleDrive内の最新10件分が表示されるので、「COPY PATH」からPATHをコピーできます。
    *   対象の動画ファイルがない場合は、直接ドライブ内のPATHを参照し、入力してください。
    *   「動画時間を含める」のチェックボックスでテキストファイルに動画時間も記述することができます。
5.   処理が終わりましたら、「処理が完了しました」と記述されます。

6.   Drive/MyDrive直下に「文字起こし」というフォルダが作られ、生成したテキストファイルを格納します。
    *   セッション切れなどによるDL漏れを防ぐためです。

↓それでは、▷をクリックし実行していきましょう

In [7]:
#@title
!pip install faster-whisper > /dev/null

import ipywidgets as widgets
from IPython.display import display, Javascript
import time
from faster_whisper import WhisperModel
import re
import os
import glob
from google.colab import drive
from IPython.display import display, HTML

# Google ドライブをマウント
drive.mount('/content/drive')

model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# Google ドライブ内の音声・動画ファイルのパスを表示
print("GoogleDrive内の音声・動画ファイル（直近10ファイル分）:")
video_extensions = ['.mp4', '.avi', '.mov', '.flv']
audio_extensions = ['.mp3', '.wav', '.flac']
file_paths = []

# 音声・動画ファイルのパスをリストアップ
for dirpath, dirnames, filenames in os.walk("/content/drive/My Drive"):
    for filename in filenames:
        ext = os.path.splitext(filename)[-1]
        if ext.lower() in video_extensions + audio_extensions:
            file_paths.append(os.path.join(dirpath, filename))

# ファイルを最終更新時間でソートし、最新の10件を取得
file_paths = sorted(file_paths, key=os.path.getmtime, reverse=True)[:10]

# 最新の10件のファイルのパスを表示
for file_path in file_paths:
    # ボタンを作成
    button = widgets.Button(description='Copy Path')
    # ボタンがクリックされたときの動作を定義
    def on_button_clicked(b, path=file_path):
        # JavaScriptを使ってクリップボードにパスをコピー
        script = '''
        function copyToClipboard(text) {{
            var dummy = document.createElement("textarea");
            document.body.appendChild(dummy);
            dummy.value = text;
            dummy.select();
            document.execCommand("copy");
            document.body.removeChild(dummy);
        }}
        '''
        script += 'copyToClipboard("{}");'.format(path)
        display(Javascript(script))
    button.on_click(on_button_clicked)
    display(widgets.HBox([widgets.Label(file_path), button]))


#動画→テキスト
def movie2txt(path):
    segments, info = model.transcribe(path, beam_size=5, language='ja')
    text = list(segments)
    match = re.search('.*/(.*?)\.', path)
    output_dir = "/content/drive/My Drive/文字起こし/"
    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir + match.group(1) + ".txt", "w", encoding="utf-8") as f:
        for segment in text:
            f.write("%s\n" % segment.text)
#動画→テキスト（時間付き）
def movie2txt_time(path):
    segments, info = model.transcribe(path, beam_size=5, language='ja')
    text = list(segments)
    match = re.search('.*/(.*?)\.', path)
    output_dir = "/content/drive/My Drive/文字起こし/"
    os.makedirs(output_dir, exist_ok=True)
    with open(output_dir + match.group(1) + ".txt", "w", encoding="utf-8") as f:
        for segment in text:
            f.write("[%.2fs -> %.2fs] %s\n" % (segment.start, segment.end, segment.text))
#ディレクトリ内の全動画を出力する
def text2movie_dir(directory, include_time):
    file_paths = glob.glob(os.path.join(directory, '*'))
    for file_path in file_paths:
        if include_time:
            movie2txt_time(file_path)
        else:
            movie2txt(file_path)

# テキストボックスを作成
text = widgets.Text(
    value='',
    placeholder='対象のPATHを入力',
    description='PATH:',
    layout=widgets.Layout(width='50%')  # ここでボックスの大きさを指定します。70%は画面幅に対する相対的なサイズです。
)


# チェックボックスを作成（時間を含めるかどうかを選択）
checkbox = widgets.Checkbox(
    value=False,
    description='動画時間を含める',
)

# ボタンを作成
button = widgets.Button(description='Convert')

# 出力ウィジェットを作成
output = widgets.Output()

# ボタンがクリックされたときの動作を定義
def on_button_clicked(b):
    path = text.value
    include_time = checkbox.value
    with output:
        output.clear_output()
        start_time = time.time()  # 処理開始時間
        if os.path.isdir(path):
    # ディレクトリ内の全動画を処理
            print("ディレクトリ内の動画を処理しています: " + path)
            text2movie_dir(path, include_time)
        elif os.path.isfile(path):
            # ファイルを処理
            print("動画を処理しています: " + path)
            if include_time:
                movie2txt_time(path)
            else:
                movie2txt(path)
        else:
            print("Invalid path: " + path)
        end_time = time.time()  # 処理終了時間
        elapsed_time = end_time - start_time  # 経過時間
        print("処理時間: {:.2f} seconds".format(elapsed_time))  # 経過時間を表示
        display(HTML('<h2 style="color: red;">処理が完了しました</h2>'))

button.on_click(on_button_clicked)

# UIを表示
display(text, checkbox, button, output)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GoogleDrive内の音声・動画ファイル（直近10ファイル分）:


Text(value='', description='PATH:', layout=Layout(width='50%'), placeholder='対象のPATHを入力')

Checkbox(value=False, description='動画時間を含める')

Button(description='Convert', style=ButtonStyle())

Output()

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 要約（開発中）

In [5]:
!pip install openai

import openai

openai.api_key = "sk-"

def system_message(text_file):
    system_message = f"""以下の会議の音声を文字起こししたものです。この内容を踏まえて、ユーザーの質問に答えなさい。

##会議音声を文字起こししたテキスト
{text_file}

テキストから、会議の要旨や議題を抑え、各議題における内容が読み取れるように箇条書きで記述してください。

会議の要旨
・～～～～
・～～～～

議題1

議題2
"""
    return system_message

#モデルを指定し、詳細設定を行います（今回はほぼデフォルト設定）
def run_conversation(text_file, user_prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        max_tokens=2000,
        messages=[
            {"role": "system", "content": system_message(text_file)},
            {"role": "user", "content": user_prompt}
        ],
    )
    return response

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.5 MB/s eta 0:00:00
